In [ ]:
import plotly.graph_objects as go

from src.projects.Chemotaxis.classes.Body import Body
from src.projects.Chemotaxis.classes.Cell import Glucose, Lactose, AminoAcid, OxygenBubble
from src.projects.Chemotaxis.classes.Bacteria import EscherichiaColy, Bacteria

In [ ]:
body = Body(cell_specs=[
    (EscherichiaColy, 1), (Glucose, 3), (Lactose, 1), (AminoAcid, 2), (OxygenBubble, 4)
])

In [ ]:
path = body.chemotaxis(1000)

In [ ]:
fig = go.Figure()

for cell, positions in path.items():
    xs, ys = zip(*positions)
    name = type(cell).__name__

    if isinstance(cell, Bacteria):
        fig.add_trace(go.Scatter(
            x=xs, y=ys, mode='lines',
            name=f"Bacteria ({name})",
            line=dict(width=3)
        ))
        # Marker Start
        fig.add_trace(go.Scatter(
            x=[xs[0]], y=[ys[0]],
            mode='markers', name="Start",
            marker=dict(color='green', size=10, symbol='circle'),
            showlegend=True  # N'afficher que la première occurrence
        ))
        # Marker End
        fig.add_trace(go.Scatter(
            x=[xs[-1]], y=[ys[-1]],
            mode='markers', name="End",
            marker=dict(color='red', size=10, symbol='x'),
            showlegend=True  # N'afficher que la première occurrence
        ))
    else:
        fig.add_trace(go.Scatter(
            x=xs, y=ys, mode='lines',
            name=f"Target ({name})",
            line=dict(dash='dash')
        ))
        # Marker Start pour les targets
        fig.add_trace(go.Scatter(
            x=[xs[0]], y=[ys[0]],
            mode='markers', name="Start",
            marker=dict(color='blue', size=8, symbol='square'),
            showlegend=False  # N'afficher pas de nouveau dans la légende
        ))
        # Marker End pour les targets
        fig.add_trace(go.Scatter(
            x=[xs[-1]], y=[ys[-1]],
            mode='markers', name="End",
            marker=dict(color='orange', size=8, symbol='x'),
            showlegend=False  # N'afficher pas de nouveau dans la légende
        ))

fig.update_layout(
    title="Chemotaxis Path of Cells",
    xaxis=dict(title="X Position", range=[-body.size, body.size]),
    yaxis=dict(title="Y Position", range=[-body.size, body.size], scaleanchor="x", scaleratio=1),
    showlegend=True,
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.show()

In [ ]:
def plot_chemotaxis(path, body, plot_trajectory=True):
    cells = list(path.keys())
    steps = len(next(iter(path.values())))  # nombre de frames = durée T
    fig = go.Figure()

    cell_colors = {
        'EscherichiaColy': 'red',
        'Glucose': 'blue',
        'Lactose': 'white',
        'AminoAcid': 'purple',
        'OxygenBubble': 'green'
    }

    # --- Initialisation des traces (pour T = 0) ---
    for cell in cells:
        name = type(cell).__name__
        x0, y0 = path[cell][0]

        symbol = 'circle-x' if isinstance(cell, Bacteria) else 'star-diamond'

        # Ligne vide pour la trajectoire qui va s'étendre avec T si plot_trajectory=True
        if plot_trajectory:
            fig.add_trace(go.Scatter(
                x=[], y=[],
                mode='lines',
                line=dict(color=cell_colors[name], width=2),
                name=f"Path - {name}",
                showlegend=True
            ))

        # Point pour la position actuelle à T=0
        fig.add_trace(go.Scatter(
            x=[x0], y=[y0],
            mode='markers',
            marker=dict(size=10, color=cell_colors[name], symbol=symbol),
            name=f"{name} Position",
            showlegend=True
        ))

    # --- Création des frames ---
    frames = []
    for t in range(steps):
        frame_data = []
        for cell in cells:
            xs, ys = zip(*path[cell])
            name = type(cell).__name__

            # Si plot_trajectory=True, on trace la trajectoire jusqu'à t
            if plot_trajectory:
                frame_data.append(go.Scatter(
                    x=xs[:t+1], y=ys[:t+1],
                    mode='lines',
                    line=dict(color=cell_colors[name], width=2),
                    showlegend=False
                ))

            symbol = 'circle-x' if isinstance(cell, Bacteria) else 'star-diamond'

            # Position actuelle (à t)
            frame_data.append(go.Scatter(
                x=[xs[t]], y=[ys[t]],
                mode='markers',
                marker=dict(size=10, color=cell_colors[name], symbol=symbol),
                showlegend=False
            ))

        frames.append(go.Frame(data=frame_data, name=str(t)))

    fig.frames = frames

    # --- Layout ---
    fig.update_layout(
        title="Chemotaxis – Trajectoires évolutives",
        xaxis=dict(title="X", range=[-body.size, body.size]),
        yaxis=dict(title="Y", scaleanchor="x", scaleratio=1, range=[-body.size, body.size]),
        updatemenus=[{
            "type": "buttons",
            "buttons": [
                {
                    "label": "Play",
                    "method": "animate",
                    "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]
                },
                {
                    "label": "Pause",
                    "method": "animate",
                    "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}]  # Corrigé ici
                }
            ]
        }],
        sliders=[{
            "steps": [{
                "method": "animate",
                "args": [[str(t)], {"mode": "immediate", "frame": {"duration": 0, "redraw": True}}],
                "label": str(t)
            } for t in range(steps)],
            "transition": {"duration": 0},
            "x": 0, "y": -0.1,
            "currentvalue": {"prefix": "T = "}
        }]
    )

    fig.show()

In [ ]:
plot_chemotaxis(path, body, plot_trajectory=False)

In [ ]:
plot_chemotaxis(path, body, plot_trajectory=True)